In [1]:
import pandas as pd, datetime, calendar
EPOCH0 = datetime.date(2025, 1, 1)        # arbitrary zero
from neo4j import GraphDatabase
from tqdm import tqdm
from math import ceil
# date = EPOCH0 + pd.to_timedelta((week-1)*7 + day-1, unit="d")
# seconds = (date - EPOCH0).days * 86400 + (trans_time//100)*3600 + (trans_time%100)*60

In [2]:
def make_ts(week, day, hhmm):
    base = (week - 1) * 7 + (day - 1)
    date = EPOCH0 + datetime.timedelta(days=base)
    secs  = (hhmm // 100) * 3600 + (hhmm % 100) * 60
    return int((date - EPOCH0).days * 86400 + secs)

In [11]:
def load_csvs(datadir="../data"):
    hh  = pd.read_csv(f"{datadir}/hh_demographic.csv")
    pr  = pd.read_csv(f"{datadir}/product.csv")
    trn = pd.read_csv(f"{datadir}/transaction_data.csv")
    trn["t"] = [make_ts(w, d, tt) for w, d, tt in zip(trn.WEEK_NO, trn.DAY, trn.TRANS_TIME)]
    return hh, pr, trn

In [15]:
def df_batches(df, size):
    """Yield successive size-row DataFrame slices."""
    n = len(df)
    for i in range(0, n, size):
        yield df.iloc[i:i + size]

In [19]:
def neo_ingest(uri, user, pwd, hh, pr, trn, batch=1000):
    driver = GraphDatabase.driver(uri, auth=(user, pwd))
    with driver.session() as s:
        # constraints
        s.run("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Consumer) REQUIRE c.hh_key IS UNIQUE")
        s.run("CREATE CONSTRAINT IF NOT EXISTS FOR (p:Product)  REQUIRE p.product_id IS UNIQUE")
        s.run("CREATE CONSTRAINT IF NOT EXISTS FOR (s:Seller)   REQUIRE s.store_id  IS UNIQUE")
        # merge consumers
        hh = hh.where(pd.notnull(hh), None)   # kill NaNs
        pr = pr.where(pd.notnull(pr), None)
        trn = trn.where(pd.notnull(trn), None)
        
        for chunk in tqdm(df_batches(hh, batch), total=ceil(len(hh)/batch), desc="Consumers"):
            s.run("""UNWIND $rows AS r MERGE (c:Consumer {hh_key:r.household_key}) SET c += apoc.map.clean(r, [], [null])""", rows=chunk.to_dict("records"))
        # merge products
        for chunk in tqdm(df_batches(pr, batch), total=ceil(len(pr)/batch), desc="Products"): 
            s.run("""UNWIND $rows AS r MERGE (p:Product {product_id:r.PRODUCT_ID}) SET p += apoc.map.clean(r, [], [null])""", rows=chunk.to_dict("records"))
        # create edges
        for chunk in tqdm(df_batches(trn, batch), total=ceil(len(trn)/batch), desc="Transactions"):
            s.run("""
                UNWIND $rows AS r
                MATCH (c:Consumer {hh_key:r.household_key})
                MATCH (p:Product  {product_id:r.PRODUCT_ID})
                MERGE (s:Seller   {store_id:r.STORE_ID})
                CREATE (c)-[:BOUGHT {
                        t:r.t, basket_id:r.BASKET_ID, qty:r.QUANTITY,
                        sales_value:r.SALES_VALUE, retail_disc:r.RETAIL_DISC,
                        coupon:r.COUPON_DISC + r.COUPON_MATCH_DISC
                    }]->(p)
                MERGE (c)-[:SHOPPED_AT {t:r.t}]->(s)""", rows=chunk.to_dict("records"))

In [12]:
import argparse, pathlib
# ap = argparse.ArgumentParser()
# ap.add_argument("--neo4j-uri", default="bolt://localhost:7474")
# ap.add_argument("--neo4j-user", default="neo4j")
# ap.add_argument("--neo4j-pass", default="test")
# ap.add_argument("--datadir",    default="data")
# ap.add_argument("--export-dir", default="exports")
# args = ap.parse_args()

hh, pr, trn = load_csvs()
# neo_ingest("bolt://localhost:7474", "neo4j", "Meniere19", hh, pr, trn)

In [22]:
neo_ingest("bolt://localhost:7687", "neo4j", "Meniere19", hh, pr, trn)

Transactions: 100%|██████████| 2596/2596 [04:17<00:00, 10.07it/s]


In [53]:
df = pd.read_csv("../data/file.csv")
df.head()

,user_id,item_id,timestamp,state_label,f1,f2,f3,f4,f5,f6,...,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172
0,0,0,0.0,0,-0.175063,-0.176678,-0.937091,-0.381926,0.0,-0.636535,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
1,1,1,36.0,0,-0.175063,-0.176678,-0.937091,-0.381926,0.0,-0.636535,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
2,1,1,77.0,0,-0.175063,-0.176678,-0.937091,-0.381926,0.0,-0.636535,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
3,2,2,131.0,0,-0.175063,-0.176678,-0.937091,-0.381926,0.0,-0.636535,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775
4,1,1,150.0,0,-0.175063,-0.176678,-0.937091,-0.381926,0.0,-0.636535,...,-0.090115,-0.096068,-0.068136,-0.060835,-0.058394,-0.062079,-0.044674,-0.050464,-0.041448,-0.038775


In [57]:
(df['state_label'][df['state_label']==0]).shape

(157257,)

In [58]:
df['state_label'].shape

(157474,)

In [45]:
## FOR MAKING THE COL NAMES

s = str()
j = 1
for i in range(172):
    d = i+j
    s = str(s + ",f{d}".format(d=d))
print(s)

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172
